In [1]:
%matplotlib inline
import os
import glob
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pathlib
from matplotlib.lines import Line2D
from PIL import Image
import arff



Bad key "lines.markeredgecolor" on line 81 in
/home/pedrolarben/.config/matplotlib/stylelib/gadfly.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
_METRICS_FILE = './files/benchmark_pool'
_METRIC = 'total_time'
_NUM_CLASSIFIER =  10

directory = './files/statistic_test/'+_METRIC+'/'


datasets_to_remove = ['bankmarketingnorm', 'airlinesnorm']
datasets_to_remove += ['breast', 'carnorm', 'covtype', 'electricity', 'higgs200k', 'kddcup99norm', 'ozone', 'pendigitis', 'pokerhand', 'spambase', 'susy100k']
classifiers_to_remove = ['keras_3_Dilated_Conv', 'keras_parallel_3_Dilated_Conv', 'MOA_SGD', 'MOA_WeightedMajorityAlgorithm']

In [3]:
classifier_labels_full = {
    'keras_parallel_3_Dilated_Conv_pooling':'PCNN',
    'sklearn_MLPClassifier':'MLP',
    'MOA_NaiveBayes':'NB',
    'MOA_SingleClassifierDrift':'SCD',
    'MOA_ASHoeffdingTree':'ASHT',
    'MOA_HoeffdingTree':'HT',
    'MOA_HoeffdingOptionTree':'HT',
    'MOA_LeveragingBag':'LBAG',
    'MOA_RandomHoeffdingTree':'RHT',
    'MOA_OzaBoostAdwin':'BO-AD',
    'MOA_OzaBoost':'BO',
    'sklearn_BernoulliNB':'BNB',
    'MOA_HoeffdingAdaptiveTree':'HAT',
    'MOA_ActiveClassifier':'AC',
    'MOA_OzaBag':'BA',
    'MOA_AdaHoeffdingOptionTree':'ADHT',
    'MOA_OzaBagAdwin':'BA-AD',
    'MOA_OzaBagASHT':'B-ASHT',
    'sklearn_PassiveAggressiveClassifier':'PAC',
    'sklearn_SGDClassifier':'SGD',
    'sklearn_Perceptron':'P',
    'MOA_DecisionStump':'DST',
    'MOA_OCBoost':'OCBO',
    'MOA_SPegasos':'SPEG',
    'MOA_MajorityClass':'MC',
    'MOA_Perceptron':'P'
}

In [4]:
# read data
metrics = pd.read_csv(_METRICS_FILE+'.csv')

# Delete datasets and classifier that we dont want
metrics = metrics[~metrics.classifier.isin(classifiers_to_remove)]
metrics = metrics[~metrics.dataset.isin(datasets_to_remove)]
metrics = metrics[metrics['instances']>1000]

metrics = metrics[metrics.classifier.isin(['MOA_ASHoeffdingTree', 'MOA_HoeffdingTree', 'MOA_LeveragingBag', 'MOA_OzaBoostAdwin', 'MOA_SingleClassifierDrift', 'keras_parallel_3_Dilated_Conv_pooling', 'sklearn_MLPClassifier', 'sklearn_SGDClassifier', 'MOA_NaiveBayes'])]

print(len(metrics.dataset.unique()), "datasets")
metrics.sample(10)

29 datasets


,dataset,classifier,instances,attributes,classes,accuracy,kappa,train_time_mean,test_time_mean,total_time_mean,train_time,test_time,total_time
1155,Mallat,MOA_OzaBoostAdwin,2400,1024,8,0.257316,0.216491,0.094228,0.017293,0.111521,22.614750,4.150252,26.765002
1927,Symbols,MOA_LeveragingBag,1020,398,6,0.910592,0.887084,0.048928,0.017297,0.066226,4.990684,1.764342,6.755027
1005,InsectWingbeatSound,MOA_OzaBoostAdwin,2200,256,11,0.643032,0.587699,0.032514,0.007370,0.039885,7.153188,1.621457,8.774645
1896,SwedishLeaf,MOA_HoeffdingTree,1125,128,15,0.824364,0.798404,0.001239,0.001030,0.002269,0.139983,0.116360,0.256343
620,FaceAll,MOA_SingleClassifierDrift,2250,131,14,0.854009,0.829490,0.000929,0.000893,0.001822,0.209077,0.200872,0.409949
756,FordA,MOA_HoeffdingTree,4921,500,2,0.554025,0.104158,0.000900,0.000188,0.001088,0.443572,0.092860,0.536432
2347,Yoga,MOA_LeveragingBag,3300,426,2,0.600928,0.208449,0.017749,0.001496,0.019245,5.857140,0.493722,6.350862
1746,ShapesAll,MOA_HoeffdingTree,1200,512,60,0.592386,0.565363,0.023298,0.020279,0.043577,2.795753,2.433502,5.229256
609,FaceAll,MOA_NaiveBayes,2250,131,14,0.854009,0.829490,0.000036,0.000942,0.000978,0.008012,0.211997,0.220009
2637,pendigits,MOA_HoeffdingTree,10992,16,10,0.875771,0.849582,0.000097,0.000074,0.000172,0.107082,0.081929,0.189011


In [5]:
metrics_table_kappa = metrics.pivot(index='dataset',columns='classifier')['kappa']
metrics_table_kappa = metrics_table_kappa.dropna()
metrics_table_kappa = metrics_table_kappa.reindex(metrics_table_kappa.mean().sort_values(ascending=False).index, axis=1)
metrics_table_kappa = metrics_table_kappa.sort_values(by=list(metrics_table_kappa.columns), ascending=False)

metrics_table = metrics.pivot(index='dataset',columns='classifier')[_METRIC]
metrics_table = metrics_table.dropna()
metrics_table = metrics_table.reindex(metrics_table_kappa.mean().sort_values(ascending=False).index, axis=1)
#metrics_table = metrics_table.sort_values(by=list(metrics_table_kappa.columns), ascending=False)
metrics_table = metrics_table.reindex(index=metrics_table_kappa.index)
metrics_table = metrics_table[metrics_table.columns[0:]]
metrics_table.columns = [classifier_labels_full[clf] for clf in metrics_table.columns]


metrics_table


,PCNN,MLP,NB,SCD,ASHT,HT,LBAG,BO-AD,SGD
dataset,,,,,,,,,
TwoPatterns,13.101814,0.469626,0.143602,0.245352,0.323179,0.299086,6.544951,2.601823,0.281698
CinCECGtorso,5.779472,1.221879,0.540160,0.993481,1.208453,1.217683,22.071268,10.364966,0.341138
TwoLeadECG,3.223873,0.091060,0.011793,0.019434,0.027181,0.030149,0.544053,0.420940,0.027231
Wafer,19.618655,0.778760,0.371527,0.251931,0.246253,0.218305,5.404364,4.231454,0.234448
pendigits,22.053120,0.779593,0.094604,0.188041,0.186575,0.189011,3.647886,2.599348,0.875312
FacesUCR,5.808412,0.216237,0.206723,0.394743,0.462462,0.470126,7.502576,6.344354,0.251160
Mallat,9.337214,1.502281,1.156474,2.237730,2.564332,2.585380,46.864588,26.765002,0.560855
FaceAll,5.790452,0.212365,0.220009,0.409949,0.479132,0.463587,7.583297,6.335840,0.252889
Symbols,2.939049,0.410105,0.131446,0.250829,0.321668,0.299731,6.755027,3.343133,0.173827


In [6]:
file_path = directory+_METRIC+'_table.csv'
if not os.path.exists(directory):
    os.makedirs(directory)
metrics_table.to_csv(file_path, sep=',', encoding='utf-8')

In [7]:
with open(file_path, "r+") as f:
    lines = f.readlines()
    #del lines[2]
    #del lines[0]
    lines[0] = lines[0].replace('dataset',' ', 1 )
    f.seek(0)
    f.truncate()
    f.writelines(lines)

In [8]:
[c for c in metrics_table.columns]

['PCNN', 'MLP', 'NB', 'SCD', 'ASHT', 'HT', 'LBAG', 'BO-AD', 'SGD']